In [ ]:
import h5py
import numpy as np
import os
import traceback # エラー発生時の詳細情報表示用

print(f"h5py version: {h5py.__version__}")
print(f"numpy version: {np.__version__}")

In [ ]:
# --- 設定 ---
FILE_NAME = "processdata.h5"

# --- ローカルファイルの存在確認 ---
if not os.path.exists(FILE_NAME):
    print(f"エラー: HDF5ファイル '{FILE_NAME}' が見つかりません。")
    print(f"このNotebookと同じディレクトリに '{FILE_NAME}' を配置してください。")
    # Jupyter Notebookでは処理をここで止めるために例外を発生させます
    raise FileNotFoundError(f"HDF5 file '{FILE_NAME}' not found. Please place it in the same directory as this notebook.")
else:
    print(f"ローカルHDF5ファイルを使用します: '{FILE_NAME}'")

In [ ]:
def print_hdf5_structure(item, indent_level=0):
    """HDF5ファイルまたはグループ内の構造を再帰的に表示する関数"""
    indent = '  ' * indent_level  # インデントの深さを調整
    if isinstance(item, h5py.File) or isinstance(item, h5py.Group):
        for key in item.keys():
            sub_item = item[key]
            print(f"{indent}├── {key}  (Type: {type(sub_item).__name__})")
            if isinstance(sub_item, h5py.Group):
                print_hdf5_structure(sub_item, indent_level + 1)
            elif isinstance(sub_item, h5py.Dataset):
                print(f"{indent}│   └── Shape: {sub_item.shape}, Dtype: {sub_item.dtype}")
                # データセットの属性も表示（もしあれば）
                if sub_item.attrs:
                    print(f"{indent}│       └── Attributes:")
                    for attr_name, attr_val in sub_item.attrs.items():
                        attr_val_str = str(attr_val)
                        # 属性値が長すぎる場合は省略
                        display_attr_val = attr_val_str[:70] + '...' if len(attr_val_str) > 70 else attr_val_str
                        print(f"{indent}│           ├── {attr_name}: {display_attr_val}")
    else:
        print(f"{indent}Error: Expected h5py.File or h5py.Group, got {type(item)}")


print(f"\n--- HDF5ファイル '{FILE_NAME}' の構造 ---")
try:
    with h5py.File(FILE_NAME, 'r') as hf: # 'r'モードで読み込み専用で開く
        print(f"{FILE_NAME} (File)")
        print_hdf5_structure(hf, indent_level=0)
except Exception as e:
    print(f"HDF5ファイルの読み込みまたは構造表示中にエラーが発生しました: {e}")
    traceback.print_exc()

In [ ]:
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# 例: 確認したいデータセットのパスを指定します。
# 上記の「HDF5ファイル の構造」の出力を参考に、実際のパスに書き換えてください。
# 例1: dataset_path_to_inspect = 'RESULTS/Zone1/fluid/vx' (Fluentからのエクスポートの典型例 - 推測)
# 例2: dataset_path_to_inspect = 'Coordinates/X_Coordinates' (別の例)
dataset_path_to_inspect = 'vertex/x' # 必ず書き換えてください！
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★


print(f"\n--- データセット '{dataset_path_to_inspect}' の内容確認 ---")
if dataset_path_to_inspect == 'PLEASE_REPLACE_WITH_ACTUAL_DATASET_PATH':
    print("警告: `dataset_path_to_inspect` が書き換えられていません。")
    print("上記のファイル構造を確認し、有効なデータセットパスを指定してください。")
else:
    try:
        with h5py.File(FILE_NAME, 'r') as hf:
            if dataset_path_to_inspect in hf:
                dataset = hf[dataset_path_to_inspect]
                print(f"データセット名 (フルパス): {dataset.name}")
                print(f"形状 (Shape): {dataset.shape}")
                print(f"データ型 (Dtype): {dataset.dtype}")
                print(f"総要素数: {dataset.size}")

                # データセットの属性を表示 (もしあれば)
                if dataset.attrs:
                    print("属性 (Attributes):")
                    for attr_name, attr_val in dataset.attrs.items():
                        attr_val_str = str(attr_val)
                        display_attr_val = attr_val_str[:70] + '...' if len(attr_val_str) > 70 else attr_val_str
                        print(f"  - {attr_name}: {display_attr_val}")
                
                # データの一部を表示 (データが非常に大きい場合は注意が必要です)
                num_preview_elements = 5 # 各次元でプレビューする要素の数を制限
                print("\nデータプレビュー:")
                if dataset.ndim == 0: # スカラーデータセット
                     print(f"  値: {dataset[()]}") # スカラー値の取得
                elif dataset.size == 0: # 空のデータセット
                    print("  データセットは空です。")
                else: # 1次元以上の配列
                    # 各次元の最初の数要素を表示するスライスを作成
                    slicing = []
                    actual_elements_shown = []
                    for dim_size in dataset.shape:
                        preview_count = min(num_preview_elements, dim_size)
                        slicing.append(slice(0, preview_count))
                        actual_elements_shown.append(preview_count)
                    
                    preview_data = dataset[tuple(slicing)]
                    print(f"  最初の約 {actual_elements_shown} 要素 (またはそれ以下) を表示:")
                    print(preview_data)

                    # (オプション) もしデータが数値型なら、基本的な統計情報を表示
                    # 注意: 全データをメモリにロードするため、巨大なデータセットでは問題が起きる可能性があります。
                    # 必要に応じてコメントを解除し、データサイズを考慮して使用してください。
                    # if np.issubdtype(dataset.dtype, np.number) and dataset.size > 0 and dataset.size < 1_000_000: # 例: 100万要素未満の場合のみ
                    #     print("\n基本的な統計情報 (全データから計算):")
                    #     try:
                    #         full_data = dataset[:] # 全データをロード
                    #         print(f"  最小値: {np.min(full_data)}")
                    #         print(f"  最大値: {np.max(full_data)}")
                    #         print(f"  平均値: {np.mean(full_data)}")
                    #         print(f"  標準偏差: {np.std(full_data)}")
                    #     except Exception as stat_e:
                    #         print(f"  統計情報の計算中にエラー: {stat_e}")
            else:
                print(f"エラー: データセット '{dataset_path_to_inspect}' がファイル内に見つかりません。")
                print("上記のファイル構造を確認し、正しいパスを指定してください。")
                print("利用可能なトップレベルのオブジェクト:", list(hf.keys()))

    except Exception as e:
        print(f"データセット '{dataset_path_to_inspect}' の読み込みまたは表示中にエラーが発生しました: {e}")
        traceback.print_exc()

In [ ]:
import pyvista as pv

# --- (前のセルで h5py, numpy, pyvista as pv, os, traceback がインポート済みと仮定) ---
# --- (FILE_NAME = "processdata.h5" も定義済みと仮定) ---

print("\n--- ステップX: HDF5データからPyVista UnstructuredGridを構築して可視化 ---")

try:
    with h5py.File(FILE_NAME, 'r') as hf:
        # 1. 節点座標の読み込みと整形
        print("節点座標を読み込み中...")
        if 'vertex/x' in hf and 'vertex/y' in hf and 'vertex/z' in hf:
            x_coords = hf['vertex/x'][:].squeeze() # .squeeze()で次元1の軸を削除 (1, N) -> (N,)
            y_coords = hf['vertex/y'][:].squeeze()
            z_coords = hf['vertex/z'][:].squeeze()

            # (N, 3) の形状のNumPy配列に結合
            points = np.vstack((x_coords, y_coords, z_coords)).T
            print(f"  読み込んだ節点数: {points.shape[0]}, 次元: {points.shape[1]}")
            if points.shape[0] != 18905 or points.shape[1] != 3:
                print(f"    警告: 期待される節点形状 (18905, 3) と異なります: {points.shape}")
        else:
            raise KeyError("必要な座標データセット (vertex/x, vertex/y, vertex/z) が見つかりません。")

        # 2. 要素の接続情報の読み込みと整形
        print("\n要素の接続情報を読み込み中...")
        if 'elements' in hf:
            # elementsデータセットは (4, 87007) の形状で、各列が1つの要素を表していると仮定
            # PyVista/VTKは各行が1つの要素を表す形式 (num_elements, num_nodes_per_element) を好むので転置(.T)
            elements_raw = hf['elements'][:].T # (87007, 4) になるはず
            print(f"  読み込んだ要素の形状 (転置後): {elements_raw.shape}")

            # データ型を整数に変換 (節点インデックスのため)
            # また、MATLAB由来の場合、インデックスが1から始まる可能性があるので、0から始まるように調整
            # ここではまず整数に変換し、インデックスの範囲を確認してから調整を検討
            elements_int = elements_raw.astype(np.int_) # または np.int64

            # 節点インデックスが0から始まるか、1から始まるかを確認 (データの最小値で判断)
            min_index = np.min(elements_int)
            if min_index == 1:
                print("  要素の節点インデックスが1から始まっていると判断し、0から始まるように調整します。")
                elements_int -= 1
            elif min_index < 0:
                print(f"  警告: 要素の節点インデックスに負の値 ({min_index}) が含まれています。データを確認してください。")
            else:
                print("  要素の節点インデックスは既に0から始まっているか、0以上のようです。")


            # PyVistaのUnstructuredGridに入力するセル配列形式を作成
            # 四面体 (tetra) の場合、各セルの前に節点数 (4) を挿入
            # VTK_TETRA のセルタイプIDは 10
            num_elements = elements_int.shape[0]
            # [4, n0,n1,n2,n3,  4, n4,n5,n6,n7, ...] という1D配列を作る
            cells_pv = np.hstack((np.full((num_elements, 1), 4, dtype=np.int_), elements_int)).flatten()
            cell_types_pv = np.full(num_elements, pv.CellType.TETRA.value, dtype=np.uint8)
            
            print(f"  PyVista用のセル配列の長さ: {len(cells_pv)}")
            print(f"  PyVista用のセルタイプ配列の長さ: {len(cell_types_pv)}")
        else:
            raise KeyError("必要な要素データセット ('elements') が見つかりません。")

        # 3. PyVista UnstructuredGridの作成
        print("\nPyVista UnstructuredGrid を作成中...")
        grid = pv.UnstructuredGrid(cells_pv, cell_types_pv, points)
        print("UnstructuredGrid の作成成功。")
        print(grid) # gridオブジェクトの情報を表示

        # 4. (オプション) スカラーデータの付加 (例: doping)
        if 'doping' in hf:
            print("\n'doping' データを節点データとして付加中...")
            doping_data = hf['doping'][:].squeeze()
            if doping_data.shape[0] == grid.n_points:
                grid.point_data['doping'] = doping_data
                print(f"  'doping' データを {grid.n_points} 個の節点に付加しました。")
            else:
                print(f"  警告: 'doping' データの要素数 ({doping_data.shape[0]}) が節点数 ({grid.n_points}) と一致しません。付加をスキップします。")
        
        # 5. PyVistaでの可視化
        print("\nPyVistaで3Dメッシュを可視化中...")
        plotter = pv.Plotter(notebook=True, window_size=[800,600]) # Jupyterバックエンドは前のセルで設定済みと仮定
        
        # 'doping' データがあれば、それで色付けして表示
        if 'doping' in grid.point_data:
            plotter.add_mesh(grid, scalars='doping', show_edges=True, cmap='viridis')
            plotter.add_scalar_bar(title='Doping Concentration')
        else:
            plotter.add_mesh(grid, show_edges=True, color='lightblue')
            
        plotter.view_isometric()
        plotter.enable_zoom_style() # enable_zoom_scaling() から修正
        plotter.show_axes()
        plotter.add_title("3D Mesh from HDF5 (processdata.h5)")
        plotter.show()

except KeyError as ke:
    print(f"キーエラー: HDF5ファイル内に期待されるデータセットが見つかりませんでした: {ke}")
    print("ファイル構造を再確認してください。")
except Exception as e:
    print(f"メッシュの構築または可視化中にエラーが発生しました: {e}")
    traceback.print_exc()